### Comparison of file origin between CDA and PDC API

The [09CO022 Example- CDA v2 notebook](https://github.com/ianfore/cdatest/blob/main/v2tests/09CO022%20Example%20-%20CDA%20v2.ipynb) showed that CDA lists the file ids of both tumor and normal samples as identical. This was true for many studies. This seems odd. A key part of the study design is likely to be a comparison of tumor and normal. One would expect at least some files to be specific to normal and to tumor. 

We should be able to use the PDC API to determine what the PDC says about these files, and confirm whether they derive from one specimen (aliquot) only, or from both tumor and normal.

First we'll define a function to process and summarize the files for a given study

In [59]:
from cdapython import Q, columns, unique_terms

def processProject(projName):
    pq = Q('ResearchSubject.associated_project = "{}"'.format(projName))
    pr = pq.run(limit=2)
    for subject2 in pr[1]['ResearchSubject']:
        subid = subject2['identifier'][0]
        if subid['system'] == 'PDC':
            print('_'*50)
            print("Subject: {}:{}".format(subid['system'],subid['value']))
            print("Specimen count: {}".format(len(subject2['Specimen'])))
            lastFileList = []
            for s in subject2['Specimen']:
                print('_'*10)
                print ('Specimen {}'.format(s['id']))
                print("Source material {}".format(s['source_material_type']))
                print ("{} derived from {}".format(s['specimen_type'],s['derived_from_specimen']))
                print ("files {}".format(len(s['File'])))
                specimenFiles = {}
                filesPrinted = False
                for f in s['File']:
                    specimenFiles[f['id']] = f['label']
                #specimenFiles.sort()
                if specimenFiles == lastFileList:
                    print("Same file content as previous specimen")
                lastFileList = specimenFiles
    return lastFileList

In [60]:
file_ids = processProject('Integrated Proteogenomic Characterization of HBV-related Hepatocellular carcinoma')

Getting results from database

Total execution time: 2136 ms
__________________________________________________
Subject: PDC:7a59ad05-1168-11ea-9bfa-0a42f3c845fe
Specimen count: 4
__________
Specimen 7a5b64a9-1168-11ea-9bfa-0a42f3c845fe
Source material Primary Tumor
sample derived from initial specimen
files 285
__________
Specimen 7a5d8c1c-1168-11ea-9bfa-0a42f3c845fe
Source material Primary Tumor
aliquot derived from 7a5b64a9-1168-11ea-9bfa-0a42f3c845fe
files 285
Same file content as previous specimen
__________
Specimen 7a5b676c-1168-11ea-9bfa-0a42f3c845fe
Source material Solid Tissue Normal
sample derived from initial specimen
files 285
Same file content as previous specimen
__________
Specimen 7a5d8e05-1168-11ea-9bfa-0a42f3c845fe
Source material Solid Tissue Normal
aliquot derived from 7a5b676c-1168-11ea-9bfa-0a42f3c845fe
files 285
Same file content as previous specimen


#### Define a function to use the PDC API to list some details about a file

In [70]:
#Get details about a single file
import requests
import json

def getPDCFileDetails(file_id):
    # The URL for our API calls
    url = 'https://pdc.cancer.gov/graphql'

    # query to get file metadata

    query = '{ fileMetadata(file_id: "'+file_id+'" acceptDUA: true) { ' + '''
        file_name
        file_size
        md5sum
        file_location
        file_submitter_id
        fraction_number
        experiment_type
        aliquots {
          aliquot_id
          aliquot_submitter_id
          sample_id
          sample_submitter_id
        }
      }
    }'''


    response = requests.post(url, json={'query': query})

    if(response.ok):
        #If the response was OK then print the returned JSON
        jData = json.loads(response.content)
        fileMD = jData['data']['fileMetadata'][0]
        #print (fileMetadata['file_name'],fileMetadata['fraction_number'],fileMetadata['experiment_type'])
        aqCount = len(fileMD['aliquots'])
        #if aqCount != 4:
        #print ("File {} has {} aliquots".format(file_id, aqCount))
        print("fraction:{} Expt type:{}".format(fileMD['fraction_number'], fileMD['experiment_type']))
        print("Linked aliquots:")
        for a in fileMD['aliquots']:
            print("sample_submitter_id:{} aliquot_id:{}".format(a['sample_submitter_id'], a['aliquot_id']))
            #print(a)

    else:
        # If response code is not ok (200), print the resulting http error code with description
        response.raise_for_status()

In [63]:
getPDCFileDetails("0431f842-6ef0-47c0-b5b2-e68d64cd9837")

fraction:12 Expt type:iTRAQ4
sample_submitter_id:Tumor Pat64 aliquot_id:e8dbfb3d-693a-11ea-b1fd-0aad30af8a83
sample_submitter_id:Tumor Pat51 aliquot_id:e8dbb9bb-693a-11ea-b1fd-0aad30af8a83
sample_submitter_id:adjacent Pat64 aliquot_id:e8daf96a-693a-11ea-b1fd-0aad30af8a83
sample_submitter_id:adjacent Pat51 aliquot_id:e8dae938-693a-11ea-b1fd-0aad30af8a83


In [64]:
for id, lb in file_ids.items():
    print("File:{}".format(lb))
    getPDCFileDetails(id)

File:20180408_HF_ZHW_total_liver26_F23.mzML.gz
fraction:23 Expt type:TMT11
sample_submitter_id:MIX aliquot_id:7a5e18c9-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:861 aliquot_id:7a5d8a89-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:862 aliquot_id:7a5d8b59-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:863 aliquot_id:7a5d8c1c-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:864 aliquot_id:7a5d8e05-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:865 aliquot_id:7a5d8ee3-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:866 aliquot_id:7a5d8fa9-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:867 aliquot_id:7a5d9175-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:868 aliquot_id:7a5d9249-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:873 aliquot_id:7a5d930b-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:874 aliquot_id:7a5d95c4-1168-11ea-9bfa-0a42f3c845fe
File:20180408_HF_ZHW_total_liver26_F01.mzid.gz
fraction:1 Expt type:TMT11
sample_submitter_id:MIX aliquot_id:7a5e18c9-1168-11ea-9bfa-

sample_submitter_id:862 aliquot_id:7a5d8b59-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:863 aliquot_id:7a5d8c1c-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:864 aliquot_id:7a5d8e05-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:865 aliquot_id:7a5d8ee3-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:866 aliquot_id:7a5d8fa9-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:867 aliquot_id:7a5d9175-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:868 aliquot_id:7a5d9249-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:873 aliquot_id:7a5d930b-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:874 aliquot_id:7a5d95c4-1168-11ea-9bfa-0a42f3c845fe
File:20180408_HF_ZHW_total_liver26_F18.mzML.gz
fraction:18 Expt type:TMT11
sample_submitter_id:MIX aliquot_id:7a5e18c9-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:861 aliquot_id:7a5d8a89-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:862 aliquot_id:7a5d8b59-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:863 aliquot_id:7a5d8c1c-1168-11ea-9bfa-0a

fraction:29 Expt type:TMT11
sample_submitter_id:MIX aliquot_id:7a5e18c9-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:861 aliquot_id:7a5d8a89-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:862 aliquot_id:7a5d8b59-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:863 aliquot_id:7a5d8c1c-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:864 aliquot_id:7a5d8e05-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:865 aliquot_id:7a5d8ee3-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:866 aliquot_id:7a5d8fa9-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:867 aliquot_id:7a5d9175-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:868 aliquot_id:7a5d9249-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:873 aliquot_id:7a5d930b-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:874 aliquot_id:7a5d95c4-1168-11ea-9bfa-0a42f3c845fe
File:20180620_fusion_zhw_TMT11_liver26_pho_F01.raw
fraction:1 Expt type:TMT11
sample_submitter_id:MIX aliquot_id:7a5e18c9-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:861 aliquo

fraction:10 Expt type:TMT11
sample_submitter_id:MIX aliquot_id:7a5e18c9-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:861 aliquot_id:7a5d8a89-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:862 aliquot_id:7a5d8b59-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:863 aliquot_id:7a5d8c1c-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:864 aliquot_id:7a5d8e05-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:865 aliquot_id:7a5d8ee3-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:866 aliquot_id:7a5d8fa9-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:867 aliquot_id:7a5d9175-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:868 aliquot_id:7a5d9249-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:873 aliquot_id:7a5d930b-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:874 aliquot_id:7a5d95c4-1168-11ea-9bfa-0a42f3c845fe
File:20180620_fusion_zhw_TMT11_liver26_pho_F11.raw
fraction:11 Expt type:TMT11
sample_submitter_id:MIX aliquot_id:7a5e18c9-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:861 aliqu

fraction:37 Expt type:TMT11
sample_submitter_id:MIX aliquot_id:7a5e18c9-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:861 aliquot_id:7a5d8a89-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:862 aliquot_id:7a5d8b59-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:863 aliquot_id:7a5d8c1c-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:864 aliquot_id:7a5d8e05-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:865 aliquot_id:7a5d8ee3-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:866 aliquot_id:7a5d8fa9-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:867 aliquot_id:7a5d9175-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:868 aliquot_id:7a5d9249-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:873 aliquot_id:7a5d930b-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:874 aliquot_id:7a5d95c4-1168-11ea-9bfa-0a42f3c845fe
File:20180620_fusion_zhw_TMT11_liver26_pho_F08.mzid.gz
fraction:8 Expt type:TMT11
sample_submitter_id:MIX aliquot_id:7a5e18c9-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:861 al

...

fraction:23 Expt type:TMT11
sample_submitter_id:MIX aliquot_id:7a5e18c9-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:861 aliquot_id:7a5d8a89-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:862 aliquot_id:7a5d8b59-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:863 aliquot_id:7a5d8c1c-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:864 aliquot_id:7a5d8e05-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:865 aliquot_id:7a5d8ee3-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:866 aliquot_id:7a5d8fa9-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:867 aliquot_id:7a5d9175-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:868 aliquot_id:7a5d9249-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:873 aliquot_id:7a5d930b-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:874 aliquot_id:7a5d95c4-1168-11ea-9bfa-0a42f3c845fe
File:20180408_HF_ZHW_total_liver26_F48.mzid.gz
fraction:48 Expt type:TMT11
sample_submitter_id:MIX aliquot_id:7a5e18c9-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:861 aliquot_i

fraction:6 Expt type:TMT11
sample_submitter_id:MIX aliquot_id:7a5e18c9-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:861 aliquot_id:7a5d8a89-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:862 aliquot_id:7a5d8b59-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:863 aliquot_id:7a5d8c1c-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:864 aliquot_id:7a5d8e05-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:865 aliquot_id:7a5d8ee3-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:866 aliquot_id:7a5d8fa9-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:867 aliquot_id:7a5d9175-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:868 aliquot_id:7a5d9249-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:873 aliquot_id:7a5d930b-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:874 aliquot_id:7a5d95c4-1168-11ea-9bfa-0a42f3c845fe
File:20180408_HF_ZHW_total_liver26_F25.mzML.gz
fraction:25 Expt type:TMT11
sample_submitter_id:MIX aliquot_id:7a5e18c9-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:861 aliquot_id

fraction:13 Expt type:TMT11
sample_submitter_id:MIX aliquot_id:7a5e18c9-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:861 aliquot_id:7a5d8a89-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:862 aliquot_id:7a5d8b59-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:863 aliquot_id:7a5d8c1c-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:864 aliquot_id:7a5d8e05-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:865 aliquot_id:7a5d8ee3-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:866 aliquot_id:7a5d8fa9-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:867 aliquot_id:7a5d9175-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:868 aliquot_id:7a5d9249-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:873 aliquot_id:7a5d930b-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:874 aliquot_id:7a5d95c4-1168-11ea-9bfa-0a42f3c845fe
File:20180408_HF_ZHW_total_liver26_F09.mzid.gz
fraction:9 Expt type:TMT11
sample_submitter_id:MIX aliquot_id:7a5e18c9-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:861 aliquot_id

fraction:16 Expt type:TMT11
sample_submitter_id:MIX aliquot_id:7a5e18c9-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:861 aliquot_id:7a5d8a89-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:862 aliquot_id:7a5d8b59-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:863 aliquot_id:7a5d8c1c-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:864 aliquot_id:7a5d8e05-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:865 aliquot_id:7a5d8ee3-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:866 aliquot_id:7a5d8fa9-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:867 aliquot_id:7a5d9175-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:868 aliquot_id:7a5d9249-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:873 aliquot_id:7a5d930b-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:874 aliquot_id:7a5d95c4-1168-11ea-9bfa-0a42f3c845fe
File:20180408_HF_ZHW_total_liver26_F21.mzid.gz
fraction:21 Expt type:TMT11
sample_submitter_id:MIX aliquot_id:7a5e18c9-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:861 aliquot_i

fraction:7 Expt type:TMT11
sample_submitter_id:MIX aliquot_id:7a5e18c9-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:861 aliquot_id:7a5d8a89-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:862 aliquot_id:7a5d8b59-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:863 aliquot_id:7a5d8c1c-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:864 aliquot_id:7a5d8e05-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:865 aliquot_id:7a5d8ee3-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:866 aliquot_id:7a5d8fa9-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:867 aliquot_id:7a5d9175-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:868 aliquot_id:7a5d9249-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:873 aliquot_id:7a5d930b-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:874 aliquot_id:7a5d95c4-1168-11ea-9bfa-0a42f3c845fe
File:20180408_HF_ZHW_total_liver26_F29.mzid.gz
fraction:29 Expt type:TMT11
sample_submitter_id:MIX aliquot_id:7a5e18c9-1168-11ea-9bfa-0a42f3c845fe
sample_submitter_id:861 aliquot_id

In [65]:
### Project: Academia Sinica LUAD-100

In [66]:
luad_file_ids = processProject('Academia Sinica LUAD-100')

Getting results from database

Total execution time: 1413 ms
__________________________________________________
Subject: PDC:24b3d98c-8beb-11ea-b1fd-0aad30af8a83
Specimen count: 4
__________
Specimen 24b53f41-8beb-11ea-b1fd-0aad30af8a83
Source material Normal Adjacent Tissue
sample derived from initial specimen
files 93
__________
Specimen 24b6674b-8beb-11ea-b1fd-0aad30af8a83
Source material Normal Adjacent Tissue
aliquot derived from 24b53f41-8beb-11ea-b1fd-0aad30af8a83
files 93
Same file content as previous specimen
__________
Specimen 24b54103-8beb-11ea-b1fd-0aad30af8a83
Source material Tumor
sample derived from initial specimen
files 93
Same file content as previous specimen
__________
Specimen 24b66826-8beb-11ea-b1fd-0aad30af8a83
Source material Tumor
aliquot derived from 24b54103-8beb-11ea-b1fd-0aad30af8a83
files 93
Same file content as previous specimen


In [67]:
for id, label in luad_file_ids.items():
    print("File:{}".format(label))
    getPDCFileDetails(id)

File:20180823_LC_TMTB25_phospho_F02_01.mzML.gz
fraction:2 Expt type:TMT10
sample_submitter_id:P099-2 aliquot_id:24b66826-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:QC2_LateStageTumor_pool aliquot_id:24b68574-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P098-1 aliquot_id:24b664a9-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P100-2 aliquot_id:24b66b27-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P099-1 aliquot_id:24b6674b-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P100-1 aliquot_id:24b668f4-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P098-2 aliquot_id:24b66573-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:QC1_(Tumor+Adjacent Normal)pool aliquot_id:24b68486-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P097-1 aliquot_id:24b6620f-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P097-2 aliquot_id:24b663d6-8beb-11ea-b1fd-0aad30af8a83
File:20180823_LC_TMTB25_phospho_F04_02.mzML.gz
fraction:4 Expt type:TMT10
sample_submitter_id:P099-2 aliquot_id:24b66826-8beb-11ea-b1f

 Expt type:TMT10
sample_submitter_id:P099-2 aliquot_id:24b66826-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:QC2_LateStageTumor_pool aliquot_id:24b68574-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P098-1 aliquot_id:24b664a9-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P100-2 aliquot_id:24b66b27-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P099-1 aliquot_id:24b6674b-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P100-1 aliquot_id:24b668f4-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P098-2 aliquot_id:24b66573-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:QC1_(Tumor+Adjacent Normal)pool aliquot_id:24b68486-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P097-1 aliquot_id:24b6620f-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P097-2 aliquot_id:24b663d6-8beb-11ea-b1fd-0aad30af8a83
File:20180823_LC_TMTB25_phospho_F01_02.raw
 Expt type:TMT10
sample_submitter_id:P099-2 aliquot_id:24b66826-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:QC2_LateStageTumor_pool aliquot_id:2

 Expt type:TMT10
sample_submitter_id:P099-2 aliquot_id:24b66826-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:QC2_LateStageTumor_pool aliquot_id:24b68574-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P098-1 aliquot_id:24b664a9-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P100-2 aliquot_id:24b66b27-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P099-1 aliquot_id:24b6674b-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P100-1 aliquot_id:24b668f4-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P098-2 aliquot_id:24b66573-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:QC1_(Tumor+Adjacent Normal)pool aliquot_id:24b68486-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P097-1 aliquot_id:24b6620f-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P097-2 aliquot_id:24b663d6-8beb-11ea-b1fd-0aad30af8a83
File:20180823_LC_TMTB25_phospho_F06_01.raw
 Expt type:TMT10
sample_submitter_id:P099-2 aliquot_id:24b66826-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:QC2_LateStageTumor_pool aliquot_id:2

fraction:1 Expt type:TMT10
sample_submitter_id:P098-1 aliquot_id:24b664a9-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P100-2 aliquot_id:24b66b27-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P100-1 aliquot_id:24b668f4-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P098-2 aliquot_id:24b66573-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:QC1_(Tumor+Adjacent Normal)pool aliquot_id:24b68486-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P099-2 aliquot_id:24b66826-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:QC2_LateStageTumor_pool aliquot_id:24b68574-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P099-1 aliquot_id:24b6674b-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P097-1 aliquot_id:24b6620f-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P097-2 aliquot_id:24b663d6-8beb-11ea-b1fd-0aad30af8a83
File:20180827_LC_TMT_Batch025_prot_F01_02.raw
fraction:1 Expt type:TMT10
sample_submitter_id:P098-1 aliquot_id:24b664a9-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P100-2 aliquo

 Expt type:TMT10
sample_submitter_id:P099-2 aliquot_id:24b66826-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:QC2_LateStageTumor_pool aliquot_id:24b68574-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P098-1 aliquot_id:24b664a9-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P100-2 aliquot_id:24b66b27-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P099-1 aliquot_id:24b6674b-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P100-1 aliquot_id:24b668f4-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P098-2 aliquot_id:24b66573-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:QC1_(Tumor+Adjacent Normal)pool aliquot_id:24b68486-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P097-1 aliquot_id:24b6620f-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P097-2 aliquot_id:24b663d6-8beb-11ea-b1fd-0aad30af8a83
File:20180827_LC_TMT_Batch025_prot_F03_01.raw
fraction:3 Expt type:TMT10
sample_submitter_id:P098-1 aliquot_id:24b664a9-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P100-2 aliquot_id:24b66

fraction:8 Expt type:TMT10
sample_submitter_id:P098-1 aliquot_id:24b664a9-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P100-2 aliquot_id:24b66b27-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P100-1 aliquot_id:24b668f4-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P098-2 aliquot_id:24b66573-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:QC1_(Tumor+Adjacent Normal)pool aliquot_id:24b68486-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P099-2 aliquot_id:24b66826-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:QC2_LateStageTumor_pool aliquot_id:24b68574-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P099-1 aliquot_id:24b6674b-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P097-1 aliquot_id:24b6620f-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P097-2 aliquot_id:24b663d6-8beb-11ea-b1fd-0aad30af8a83
File:20180827_LC_TMT_Batch025_prot_F09_01.raw
fraction:9 Expt type:TMT10
sample_submitter_id:P098-1 aliquot_id:24b664a9-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P100-2 aliquo

fraction:14 Expt type:TMT10
sample_submitter_id:P098-1 aliquot_id:24b664a9-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P100-2 aliquot_id:24b66b27-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P100-1 aliquot_id:24b668f4-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P098-2 aliquot_id:24b66573-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:QC1_(Tumor+Adjacent Normal)pool aliquot_id:24b68486-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P099-2 aliquot_id:24b66826-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:QC2_LateStageTumor_pool aliquot_id:24b68574-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P099-1 aliquot_id:24b6674b-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P097-1 aliquot_id:24b6620f-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P097-2 aliquot_id:24b663d6-8beb-11ea-b1fd-0aad30af8a83
File:20180827_LC_TMT_Batch025_prot_F15_01.raw
fraction:15 Expt type:TMT10
sample_submitter_id:P098-1 aliquot_id:24b664a9-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P100-2 aliq

fraction:18 Expt type:TMT10
sample_submitter_id:P098-1 aliquot_id:24b664a9-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P100-2 aliquot_id:24b66b27-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P100-1 aliquot_id:24b668f4-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P098-2 aliquot_id:24b66573-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:QC1_(Tumor+Adjacent Normal)pool aliquot_id:24b68486-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P099-2 aliquot_id:24b66826-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:QC2_LateStageTumor_pool aliquot_id:24b68574-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P099-1 aliquot_id:24b6674b-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P097-1 aliquot_id:24b6620f-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P097-2 aliquot_id:24b663d6-8beb-11ea-b1fd-0aad30af8a83
File:20180827_LC_TMT_Batch025_prot_F19_01.raw
fraction:19 Expt type:TMT10
sample_submitter_id:P098-1 aliquot_id:24b664a9-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P100-2 aliq

fraction:23 Expt type:TMT10
sample_submitter_id:P098-1 aliquot_id:24b664a9-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P100-2 aliquot_id:24b66b27-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P100-1 aliquot_id:24b668f4-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P098-2 aliquot_id:24b66573-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:QC1_(Tumor+Adjacent Normal)pool aliquot_id:24b68486-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P099-2 aliquot_id:24b66826-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:QC2_LateStageTumor_pool aliquot_id:24b68574-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P099-1 aliquot_id:24b6674b-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P097-1 aliquot_id:24b6620f-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P097-2 aliquot_id:24b663d6-8beb-11ea-b1fd-0aad30af8a83
File:20180827_LC_TMT_Batch025_prot_F23_02.raw
fraction:23 Expt type:TMT10
sample_submitter_id:P098-1 aliquot_id:24b664a9-8beb-11ea-b1fd-0aad30af8a83
sample_submitter_id:P100-2 aliq

In [68]:
gtown_file_ids = processProject('Georgetown Lung Cancer Proteomics Study')

Getting results from database

Total execution time: 1230 ms
__________________________________________________
Subject: PDC:9e8e8bb4-d732-11ea-b1fd-0aad30af8a83
Specimen count: 4
__________
Specimen 9e8ea492-d732-11ea-b1fd-0aad30af8a83
Source material Primary Tumor
sample derived from initial specimen
files 47
__________
Specimen 9e8ec102-d732-11ea-b1fd-0aad30af8a83
Source material Primary Tumor
aliquot derived from 9e8ea492-d732-11ea-b1fd-0aad30af8a83
files 47
Same file content as previous specimen
__________
Specimen 9e8ea66a-d732-11ea-b1fd-0aad30af8a83
Source material Solid Tissue Normal
sample derived from initial specimen
files 47
Same file content as previous specimen
__________
Specimen 9e8ec1db-d732-11ea-b1fd-0aad30af8a83
Source material Solid Tissue Normal
aliquot derived from 9e8ea66a-d732-11ea-b1fd-0aad30af8a83
files 47
Same file content as previous specimen


In [71]:
n = 0
for id, lb in gtown_file_ids.items():
    n += 1
    print(n,"_"*50)
    print("File:{}".format(lb))
    getPDCFileDetails(id)

1 __________________________________________________
File:Ctrl_4-set_2-label_113-frac_1-F1.wiff
fraction:1 Expt type:iTRAQ8
Linked aliquots:
sample_submitter_id:ICBI-000010-02 aliquot_id:9e8ecd06-d732-11ea-b1fd-0aad30af8a83
sample_submitter_id:ReferenceMix aliquot_id:9e8ecf0b-d732-11ea-b1fd-0aad30af8a83
sample_submitter_id:ICBI-000005-02 aliquot_id:9e8ec029-d732-11ea-b1fd-0aad30af8a83
sample_submitter_id:ICBI-000005-01 aliquot_id:9e8ebf40-d732-11ea-b1fd-0aad30af8a83
sample_submitter_id:ICBI-000004-02 aliquot_id:9e8ebd7a-d732-11ea-b1fd-0aad30af8a83
sample_submitter_id:ICBI-000004-01 aliquot_id:9e8ebc9e-d732-11ea-b1fd-0aad30af8a83
sample_submitter_id:ICBI-000006-02 aliquot_id:9e8ec1db-d732-11ea-b1fd-0aad30af8a83
sample_submitter_id:ICBI-000006-01 aliquot_id:9e8ec102-d732-11ea-b1fd-0aad30af8a83
2 __________________________________________________
File:Ctrl_4-set_2-label_113-frac_1-F1.wiff.scan
fraction:1 Expt type:iTRAQ8
Linked aliquots:
sample_submitter_id:ICBI-000010-02 aliquot_id:9e8ec

fraction:18 Expt type:iTRAQ8
Linked aliquots:
sample_submitter_id:ICBI-000010-02 aliquot_id:9e8ecd06-d732-11ea-b1fd-0aad30af8a83
sample_submitter_id:ReferenceMix aliquot_id:9e8ecf0b-d732-11ea-b1fd-0aad30af8a83
sample_submitter_id:ICBI-000005-02 aliquot_id:9e8ec029-d732-11ea-b1fd-0aad30af8a83
sample_submitter_id:ICBI-000005-01 aliquot_id:9e8ebf40-d732-11ea-b1fd-0aad30af8a83
sample_submitter_id:ICBI-000004-02 aliquot_id:9e8ebd7a-d732-11ea-b1fd-0aad30af8a83
sample_submitter_id:ICBI-000004-01 aliquot_id:9e8ebc9e-d732-11ea-b1fd-0aad30af8a83
sample_submitter_id:ICBI-000006-02 aliquot_id:9e8ec1db-d732-11ea-b1fd-0aad30af8a83
sample_submitter_id:ICBI-000006-01 aliquot_id:9e8ec102-d732-11ea-b1fd-0aad30af8a83
13 __________________________________________________
File:Ctrl_6-set_2-label_115-frac_1-F3.wiff
fraction:3 Expt type:iTRAQ8
Linked aliquots:
sample_submitter_id:ICBI-000010-02 aliquot_id:9e8ecd06-d732-11ea-b1fd-0aad30af8a83
sample_submitter_id:ReferenceMix aliquot_id:9e8ecf0b-d732-11ea-b1fd

fraction:20 Expt type:iTRAQ8
Linked aliquots:
sample_submitter_id:ICBI-000010-02 aliquot_id:9e8ecd06-d732-11ea-b1fd-0aad30af8a83
sample_submitter_id:ReferenceMix aliquot_id:9e8ecf0b-d732-11ea-b1fd-0aad30af8a83
sample_submitter_id:ICBI-000005-02 aliquot_id:9e8ec029-d732-11ea-b1fd-0aad30af8a83
sample_submitter_id:ICBI-000005-01 aliquot_id:9e8ebf40-d732-11ea-b1fd-0aad30af8a83
sample_submitter_id:ICBI-000004-02 aliquot_id:9e8ebd7a-d732-11ea-b1fd-0aad30af8a83
sample_submitter_id:ICBI-000004-01 aliquot_id:9e8ebc9e-d732-11ea-b1fd-0aad30af8a83
sample_submitter_id:ICBI-000006-02 aliquot_id:9e8ec1db-d732-11ea-b1fd-0aad30af8a83
sample_submitter_id:ICBI-000006-01 aliquot_id:9e8ec102-d732-11ea-b1fd-0aad30af8a83
24 __________________________________________________
File:Reference-set_2-label_121-frac_1-F8.wiff
fraction:8 Expt type:iTRAQ8
Linked aliquots:
sample_submitter_id:ICBI-000010-02 aliquot_id:9e8ecd06-d732-11ea-b1fd-0aad30af8a83
sample_submitter_id:ReferenceMix aliquot_id:9e8ecf0b-d732-11ea-b

fraction:21 Expt type:iTRAQ8
Linked aliquots:
sample_submitter_id:ICBI-000010-02 aliquot_id:9e8ecd06-d732-11ea-b1fd-0aad30af8a83
sample_submitter_id:ReferenceMix aliquot_id:9e8ecf0b-d732-11ea-b1fd-0aad30af8a83
sample_submitter_id:ICBI-000005-02 aliquot_id:9e8ec029-d732-11ea-b1fd-0aad30af8a83
sample_submitter_id:ICBI-000005-01 aliquot_id:9e8ebf40-d732-11ea-b1fd-0aad30af8a83
sample_submitter_id:ICBI-000004-02 aliquot_id:9e8ebd7a-d732-11ea-b1fd-0aad30af8a83
sample_submitter_id:ICBI-000004-01 aliquot_id:9e8ebc9e-d732-11ea-b1fd-0aad30af8a83
sample_submitter_id:ICBI-000006-02 aliquot_id:9e8ec1db-d732-11ea-b1fd-0aad30af8a83
sample_submitter_id:ICBI-000006-01 aliquot_id:9e8ec102-d732-11ea-b1fd-0aad30af8a83
36 __________________________________________________
File:Tumor_6-set_2-label_118-frac_1-F6.wiff
fraction:6 Expt type:iTRAQ8
Linked aliquots:
sample_submitter_id:ICBI-000010-02 aliquot_id:9e8ecd06-d732-11ea-b1fd-0aad30af8a83
sample_submitter_id:ReferenceMix aliquot_id:9e8ecf0b-d732-11ea-b1f

fraction:23 Expt type:iTRAQ8
Linked aliquots:
sample_submitter_id:ICBI-000010-02 aliquot_id:9e8ecd06-d732-11ea-b1fd-0aad30af8a83
sample_submitter_id:ReferenceMix aliquot_id:9e8ecf0b-d732-11ea-b1fd-0aad30af8a83
sample_submitter_id:ICBI-000005-02 aliquot_id:9e8ec029-d732-11ea-b1fd-0aad30af8a83
sample_submitter_id:ICBI-000005-01 aliquot_id:9e8ebf40-d732-11ea-b1fd-0aad30af8a83
sample_submitter_id:ICBI-000004-02 aliquot_id:9e8ebd7a-d732-11ea-b1fd-0aad30af8a83
sample_submitter_id:ICBI-000004-01 aliquot_id:9e8ebc9e-d732-11ea-b1fd-0aad30af8a83
sample_submitter_id:ICBI-000006-02 aliquot_id:9e8ec1db-d732-11ea-b1fd-0aad30af8a83
sample_submitter_id:ICBI-000006-01 aliquot_id:9e8ec102-d732-11ea-b1fd-0aad30af8a83
47 __________________________________________________
File:Tumor_7-set_2-label_119-frac_3-F23.wiff.scan
fraction:23 Expt type:iTRAQ8
Linked aliquots:
sample_submitter_id:ICBI-000010-02 aliquot_id:9e8ecd06-d732-11ea-b1fd-0aad30af8a83
sample_submitter_id:ReferenceMix aliquot_id:9e8ecf0b-d732-1

These illustrate

* The samples amalyzed on the mass spectrometer
It is possible but would be surprizing

If the latter then those scientists who in particular advocate for reproducility 

Many PDC files likely genuinely derive from multiple samples
The errors elsewhere lead to a lack of confidence that it is accurate